In [1]:
# Imports
from pyspark.sql.functions import (
    col, lit, when, trim, coalesce, create_map, array, lower,
    year, month, dayofmonth, dayofweek, weekofyear, quarter, date_format,
    avg, count, lag, sum, concat_ws, lpad, to_date, lit, to_timestamp, expr
)

from pyspark.sql.window import Window
from pyspark.sql.functions import when
import matplotlib.pyplot as plt

# Load Gold tables
df_fact_sales = spark.read.table("Projeto_II_Gold_.Projeto_II_Gold_.fact_sales")
df_dim_date = spark.read.table("Projeto_II_Gold_.Projeto_II_Gold_.dim_date")
df_dim_product = spark.read.table("Projeto_II_Gold_.Projeto_II_Gold_.dim_product")
df_dim_stores = spark.read.table("Projeto_II_Gold_.Projeto_II_Gold_.dim_stores")

# Join fact_sales with dim_date to extract year and week information
sales = df_fact_sales.join(df_dim_date, on="date_id", how="left")

# Aggregate data by week, store and product
weekly_sales = sales.groupBy("store_id", "product_id", year("full_date").alias("year"), weekofyear("full_date").alias("week")) \
    .agg(
        avg("sales").alias("weekly_sales"),
        avg("stock").alias("avg_stock"),
        avg("price").alias("avg_price"),
        avg(when(col("promo_bin_1") == "1", 1).otherwise(0)).alias("promo_bin_1_rate"),
        avg(when(col("promo_bin_2") == "1", 1).otherwise(0)).alias("promo_bin_2_rate"),
        count("sales").alias("num_obs"),
        avg(col("is_holiday").cast("int")).alias("holiday_rate"),
        avg(col("is_weekend").cast("int")).alias("weekend_rate")
    )

# Create lag features (previous 1, 2 and 3 week sales)
window_spec = Window.partitionBy("store_id", "product_id").orderBy("year", "week")
weekly_sales = weekly_sales \
    .withColumn("lag_sales_1w", lag("weekly_sales", 1).over(window_spec)) \
    .withColumn("lag_sales_2w", lag("weekly_sales", 2).over(window_spec)) \
    .withColumn("lag_sales_3w", lag("weekly_sales", 3).over(window_spec))

# Drop NA values on lag_sales
weekly_sales = weekly_sales.dropna(subset=["lag_sales_1w", "lag_sales_2w", "lag_sales_3w"])

# Join with product and store dimensions
df_features = weekly_sales \
    .join(df_dim_product, on="product_id", how="left") \
    .join(df_dim_stores, on="store_id", how="left")

# Calculate the Monday of the given year and week
df_features = df_features.withColumn(
    "week_start",
    expr("date_add(make_date(year, 1, 1), (week - 1) * 7 - pmod(datediff(make_date(year, 1, 1), '1900-01-01') + 1, 7))")
)

# Save final features table to ML_Area_
df_features.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("ML_Area_.features.features_table")

print("✅ Features table successfully created using Gold layer.")

StatementMeta(, 14bb4906-0059-4901-b4e7-e2b832012d58, 3, Finished, Available, Finished)

✅ Features table successfully created using Gold layer.


In [2]:
df_features = spark.read.table("ML_Area_.features.features_table")
features_table = df_features.toPandas()
features_table.head()

StatementMeta(, 14bb4906-0059-4901-b4e7-e2b832012d58, 4, Finished, Available, Finished)

,store_id,product_id,year,week,weekly_sales,avg_stock,avg_price,promo_bin_1_rate,promo_bin_2_rate,num_obs,...,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,storetype_id,store_size,city_name,week_start
0,S0002,P0001,2017,4,0.0,1.0,6.5,0.0,0.0,7,...,cluster_5,H01,H0105,H010501,H01050100,H0105010006,ST04,39,Adana,2017-01-22
1,S0002,P0001,2017,5,0.0,1.0,6.5,0.0,0.0,7,...,cluster_5,H01,H0105,H010501,H01050100,H0105010006,ST04,39,Adana,2017-01-29
2,S0002,P0001,2017,6,0.0,1.0,6.5,0.0,0.0,7,...,cluster_5,H01,H0105,H010501,H01050100,H0105010006,ST04,39,Adana,2017-02-05
3,S0002,P0001,2017,7,0.0,1.0,6.5,0.0,0.0,7,...,cluster_5,H01,H0105,H010501,H01050100,H0105010006,ST04,39,Adana,2017-02-12
4,S0002,P0001,2017,8,0.0,1.0,6.5,0.0,0.0,7,...,cluster_5,H01,H0105,H010501,H01050100,H0105010006,ST04,39,Adana,2017-02-19


In [3]:
features_table.dtypes

StatementMeta(, 14bb4906-0059-4901-b4e7-e2b832012d58, 5, Finished, Available, Finished)

store_id             object
product_id           object
year                  int32
week                  int32
weekly_sales        float64
avg_stock           float64
avg_price           float64
promo_bin_1_rate    float64
promo_bin_2_rate    float64
num_obs               int64
holiday_rate        float64
weekend_rate        float64
lag_sales_1w        float64
lag_sales_2w        float64
lag_sales_3w        float64
product_length      float64
product_depth       float64
product_width       float64
cluster_id           object
hierarchy1_id        object
hierarchy2_id        object
hierarchy3_id        object
hierarchy4_id        object
hierarchy5_id        object
storetype_id         object
store_size            int32
city_name            object
week_start           object
dtype: object